Assignment 1 - Building a Random Forest
This is a skeleton of a Random Forest classifier.



In [1]:
import csv
import math
from statistics import median, mode, mean
from collections import Counter
from enum import Enum
import numpy as np
import pandas as pd
import statistics
from math import log

Some simple type definitions.

In [2]:
class AttrType(Enum):
    cat = 0  # categorical (qualitative) attribute
    num = 1  # numerical (quantitative) attribute
    target = 2  # target label

class NodeType(Enum):
    root = 0
    internal = 1
    leaf = 2

class SplitType(Enum):
    bin = 0  # binary split
    multi = 1  # multi-way split

Also, some basic classes to represent an attribute, a spltting procedure, and a node.

In [3]:
class Attribute(object):
    def __init__(self, label, type):
        assert type in AttrType
        self.label = label
        self.type = type
        self.stat = None  # holds mean for numerical and mode for categorical attributes


class Splitting(object):
    def __init__(self, attr, infogain, split_type, cond, splits):
        self.attr = attr  # attribute ID (index in ATTR)
        self.infogain = infogain  # information gain if splitting is done on this attribute
        self.split_type = split_type  # one of SplitType
        self.cond = cond  # splitting condition, i.e., values on outgoing edges
        self.splits = splits  # list of training records (IDs) for each slitting condition


class Node(object):
    def __init__(self, id, type, parent_id, children=None, edge_value=None, val=None, split_type=None, split_cond=None,
                 infogain=None):
        self.id = id  # ID (same as the index in DT.model list)
        self.type = type  # one of NodeType
        self.parent_id = parent_id  # ID of parent node (None if root)
        self.children = children  # list of IDs of child nodes
        self.edge_value = edge_value  # the value of the incoming edge (only if not root node)
        self.val = val  # if root or internal node: the attribute that is compared at that node; if leaf node: the target value
        self.split_type = split_type  # one of SplitType
        self.split_cond = split_cond  # splitting condition (median value for binary splits on numerical values; otherwise a list of categorical values (corresponding to child nodes))
        self.infogain = infogain

    def append_child(self, node_id):
        self.children.append(node_id)

In [4]:
df = pd.read_csv('housing_price_train.csv')
df['LotFrontage'] = df['LotFrontage'].fillna(df['LotFrontage'].median())
df.set_index('Id')
df.to_csv('housing_price_train1.csv', index=False)

In [5]:
ATTR = []
INFILE = "housing_price_train1.csv"

for col in df.columns:
    if col == "SalePrice":
        ATTR.append(Attribute(col, AttrType.target))
    elif col == "MSSubClass" or col == "OverallQual" or col == "OverallCond":
        ATTR.append(Attribute(col, AttrType.cat))
    elif df[col].unique().dtype == "int64" or df[col].unique().dtype == "float64":
        ATTR.append(Attribute(col, AttrType.num))        
    else:
        ATTR.append(Attribute(col, AttrType.cat))

IDX_TARGET = len(ATTR) - 1 

In [6]:
# Could make a new df with only colums with numerical values and categories
numerical_df = pd.DataFrame()
for col in df.columns:
    if df[col].dtype == "int64":
        numerical_df[col] = df[col]
        continue
    else:
        df[col] = df[col].astype("category")
        new_col = '{val}_Cat'.format(val=col)
        numerical_df[new_col] = df[col].cat.codes
        
        # df[col] = df[col].astype("category")
        # new_col = '{val}_Cat'.format(val=col)
        # df[new_col] = df[col].cat.codes
numerical_df

,Id,MSSubClass,MSZoning_Cat,LotFrontage_Cat,LotArea,Street_Cat,Alley_Cat,LotShape_Cat,LandContour_Cat,Utilities_Cat,...,PoolArea,PoolQC_Cat,Fence_Cat,MiscFeature_Cat,MiscVal,MoSold,YrSold,SaleType_Cat,SaleCondition_Cat,SalePrice
0,1,60,3,36,8450,1,-1,3,3,0,...,0,-1,-1,-1,0,2,2008,8,4,208500
1,2,20,3,51,9600,1,-1,3,3,0,...,0,-1,-1,-1,0,5,2007,8,4,181500
2,3,60,3,39,11250,1,-1,0,3,0,...,0,-1,-1,-1,0,9,2008,8,4,223500
3,4,70,3,31,9550,1,-1,0,3,0,...,0,-1,-1,-1,0,2,2006,8,0,140000
4,5,60,3,55,14260,1,-1,0,3,0,...,0,-1,-1,-1,0,12,2008,8,4,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,3,33,7917,1,-1,3,3,0,...,0,-1,-1,-1,0,8,2007,8,4,175000
1456,1457,20,3,56,13175,1,-1,3,3,0,...,0,-1,2,-1,0,2,2010,8,4,210000
1457,1458,70,3,37,9042,1,-1,3,3,0,...,0,-1,0,2,2500,5,2010,8,4,266500
1458,1459,20,3,39,9717,1,-1,3,3,0,...,0,-1,-1,-1,0,4,2010,8,4,142125


A class DT representing the decision tree classifier. It could represent with methods:

a given impurity measure;
the search for the best attribute to split with;
the addition of a node to the tree;
a convenient model printer;
the recursive call for obtaining a tree;
a builder and an applier.

In [10]:
class DT(object):
    def __init__(self):
        self.data = None  # training data set (loaded into memory)
        self.model = None  # decision tree model
        # self.default_target = 0.0  # default target class HVA ER DET FOR NOE?

    def __load_data(self):
        # with open(INFILE) as csvfile:
        #     self.data = []
        #     csvreader = csv.reader(csvfile, delimiter=',')
        #     next(csvreader)
        #     for row in csvreader:
        #         rec = []
        #         for i in range(len(ATTR)):
        #             val = row[i].strip()
        #             # convert numerical attributes
        #             if ATTR[i].type == AttrType.num:  # Note that this will break for "?" (missing attribute)
        #                 if val == "":
        #                     val = None
        #                 else:
        #                     val = float(val)
        #             rec.append(val)
        #         self.data.append(rec)
        #     # print(self.data)
        #         # self.data.append([element.strip() for element in row])  # strip spaces
        self.data = numerical_df.values.tolist()
                
    def __median(self, a):
        all_attribute_values = []
        for idx in range(len(self.data)):
            all_attribute_values.append(self.data[idx][a])
        return statistics.median(all_attribute_values)
    
    
    def __split_cat(self, a, split_cond):
        """
        :return: returns the id of splitted values
        """
        children = []
        for val in split_cond:
            child = []
            for idx in range(len(self.data)):
                if self.data[idx][a] == val:
                    child.append(self.data[idx])
            if len(child) != 0:
                children.append(child)
        return children

    def __split_num(self, a, splitting_point):
        """
        :return: returns the id of splitted values 
        """
        children = [[x for x in self.data if x[a] <= splitting_point],
                    [x for x in self.data if x[a] > splitting_point]]
        return children
       

        
    def __find_best_attr(self, attrs, records):
        """
        Finds the attribute with the largest gain.

        :param attrs: Set of attributes
        :param records: Training set (list of record ids)
        :return:
        """
        # mse_p = self.__mean_squared_error(records)  # parent's MSE HVORFOR TRENGER VI DEN?
        splittings = []  # holds the splitting information for each attribute

        for a in attrs:
            assert ATTR[a].type in AttrType
            splits = {}  # record IDs corresponding to each split
            # splitting condition depends on the attribute type
            if ATTR[a].type == AttrType.target:  # skip target attribute
                continue
            elif ATTR[a].type == AttrType.cat:  # categorical attribute
                # multi-way split on each possible value
                split_mode = SplitType.multi
                # each possible attr value corresponds to a split (indexed with categorical labels)
                # Note: it's important to consider attr values from the entire training set
                split_cond = set([self.data[idx][a] for idx in range(len(self.data))])

                # TODO collect training records for each split 
                # `splits[val]` holds a list of records for a given split,
                # where `val` is an element of `split_cond`
                children = self.__split_cat(a, split_cond)
                for i, val in enumerate(split_cond):
                    splits[val] = children[i]
                
            elif ATTR[a].type == AttrType.num:  # numerical attribute => binary split on median value
                split_mode = SplitType.bin
                split_cond = self.__median(a)  # (i.e., if less or equal than this value)
                split_cond_list = []
                split_cond_list.append("{}.1".format(split_cond))
                split_cond_list.append("{}.2".format(split_cond))
                
                # TODO collect training records for each split (in `splits`)
                children = self.__split_num(a, split_cond)
                for i, val in enumerate(split_cond_list):
                    splits[val] = children[i]
            
            # TODO compute gain for attribute a
            #print(splits)
            
            infogain = 0
            datalist = []
            target = []
            for idx in range(len(self.data)):
                datalist.append(self.data[idx][a])
                target.append(self.data[idx][-1])
            target_df = pd.DataFrame(target, columns=["SalePrice"])
            datalist_df = pd.DataFrame(datalist)
            #data_df = pd.DataFrame(self.data)
            
            #data_split = self.__split(numerical_df, datalist_df[0])

            # split_labels = []
            # for key in splits:
            #     child = []
            #     for id in splits[key]:
            #         for idx in range(len(self.data)):
            #             if self.data[idx][0] == id:
            #                 child.append(self.data[idx][-1])
            #         if len(child) != 0:
            #             split_labels.append(child)
            
            split_labels = []
            for key in splits:
                child = []
                for row in splits[key]:
                    for idx in range(len(self.data)):
                        if self.data[idx] == row:
                            child.append(self.data[idx])
                    if len(child) != 0:
                        split_labels.append(child)
            
            
            
            # split_labels = [dataframe['SalePrice'] for dataframe in data_split]
            infogain = self.__calculate_information_gain(target_df, split_labels)
            print(infogain)
            # p = len(y_true) / len(y)
            # entropy = calculate_entropy(y)
            # info_gain = entropy - p*calculate_entropy(y_true) - (1-p)*calculate_entropy(y_false)
            splitting = Splitting(a, infogain, split_mode, split_cond, splits)
            splittings.append(splitting)

        # find best splitting
        best_splitting = sorted(splittings, key=lambda x: x.infogain, reverse=True)[0]
        return best_splitting
    
#########################################################################################
########################## Funksjoner til å kalkulere infogain ##########################    
#########################################################################################    
    def __calculate_entropy(self, labels):
        n_labels = len(labels)
        if n_labels <= 1:
            return 0
        value, counts = np.unique(labels, return_counts=True)
        probs = counts/n_labels
        n_classes = len(value)
        if n_classes <= 1:
            return 0
        entropy = 0
        for i in probs:
            entropy -= i*log(i,2)
        return entropy
    def __calculate_information_gain(self, target, split):
        info_gain = self.__calculate_entropy(target)
        for branch in split:
            p = len(branch)/len(target)
            info_gain -= p*self.__calculate_entropy(branch)
        return info_gain
#########################################################################################
#########################################################################################

    def __add_node(self, parent_id, node_type=NodeType.internal, edge_value=None, val=None, split_type=None,
                   split_cond=None):
        """
        Adds a node to the decision tree.

        :param parent_id:
        :param node_type:
        :param edge_value:
        :param val:
        :param split_type:
        :param split_cond:
        :return:
        """
        node_id = len(self.model)  # id of the newly assigned node
        if not self.model:  # the tree is empty
            node_type = NodeType.root

        node = Node(node_id, node_type, parent_id, children=[], edge_value=edge_value, val=val, split_type=split_type,
                    split_cond=split_cond)
        self.model.append(node)

        # also add it as a child of the parent node
        if parent_id is not None:
            self.model[parent_id].append_child(node_id)

        return node_id

    def __id3(self, attrs, records, parent_id=None, value=None):
        """
        Function ID3 that returns a decision tree.

        :param attrs: Set of attributes
        :param records: Training set (list of record ids)
        :param parent_id: ID of parent node
        :param value: Value corresponding to the parent attribute, i.e., label of the edge on which we arrived to this node
        :return:
        """
        # empty training set or empty set of attributes => create leaf node with default class
        if not records or not attrs:
            self.__add_node(parent_id, node_type=NodeType.leaf, edge_value=value, val=self.default_class)
            return

        # if all records have the same target value => create leaf node with that target value
        same = all(self.data[idx][IDX_TARGET] == self.data[records[0]][IDX_TARGET] for idx in records)
        if same:
            target = self.data[records[0]][IDX_TARGET]
            self.__add_node(parent_id, node_type=NodeType.leaf, edge_value=value, val=target)
            return

        # find the attribute with the largest gain
        splitting = self.__find_best_attr(attrs, records)
        # add node
        node_id = self.__add_node(parent_id, edge_value=value, val=splitting.attr, split_type=splitting.split_type,
                                  split_cond=splitting.cond)
        # TODO call tree construction recursively for each split
        node = self.model[node_id]
        for n in node.children:
            self.__id3(attrs, records, node_id, node.val)
        return self.model


    def print_model(self, node_id=0, level=0):
        node = self.model[node_id]
        indent = "  " * level
        if node.type == NodeType.leaf:
            print(indent + str(node.edge_value) + " [Leaf node] class=" + node.val)
        else:
            cond = " <= " + str(node.split_cond) if ATTR[node.val].type == AttrType.num else " == ? "
            if node.type == NodeType.root:
                print("[Root node] '" + ATTR[node.val].label + "'" + cond)
            else:
                print(indent + str(node.edge_value) + " [Internal node] '" + ATTR[node.val].label + "'" + cond)
            # print tree for child notes recursively
            for n_id in node.children:
                self.print_model(n_id, level + 1)

    def build_model(self):
        self.__load_data()
        self.model = []  # holds the decision tree model, represented as a list of nodes
        # Get majority class
        #   Note: Counter returns a dictionary, most_common(x) returns a list with the x most common elements as
        #         (key, count) tuples; we need to take the first element of the list and the first element of the tuple
        # self.default_target = mean([x[IDX_TARGET] for x in self.data])
        target_list = []
        for x in self.data:
            target_list.append(x[IDX_TARGET])
        # self.default_target = [int(n) for n in target_list] TRENGER VI DEN??
        self.__id3(set(range(len(ATTR) - 1)), list(range(len(self.data))))

    def apply_model(self, record):
        node = self.model[0]
        while node.type != NodeType.leaf:
        # TODO based on the value of the record's attribute that is tested in `node`,
        # set `node` to one of its child nodes until a leaf node is reached
            #print(node)
            continue
        return node.val
         
    def predict(self, records):
        predictions = []
        for record in records:
            print(record)
            pred_val = self.apply_model(record)
            # TODO append pred_val to predictions
            predictions.append(pred_val)
        return predictions

In [11]:
dt = DT()
dt.build_model()
#dt.print_model()

61865.0776181397
27123.316182214632
77585.06106192213
64981.98123873373
62543.831846425164
118696.00192731823
105566.0248108141
64652.45413795126
97536.44854596244
119537.66299955222
68728.34844471519
108167.05521978132
10791.571915342896
90495.98831645996
117335.54603094945
84662.45003356428
46470.937471372185
28180.056293839483
45971.2375959991
64038.61035737211
63642.34437519846
80155.02918189383
115930.22467698922
26699.346554847838
25776.95945103658
58712.800399809326
65867.36967845165
64604.93297184184
93913.48625576231
50280.49489118494
49459.02647016316
95977.84285120723
57967.68445798317
28155.103140472627
62386.5390001006
89899.24620617018
97594.20416948183
62245.405485137315
63032.46594854404
114128.38929484958
46607.40326800795
104043.36761229341
99280.33056898405
63049.03888751992
64813.354358783625
115742.97556162337
63106.76893165249
64476.32224363471
107702.48785163763
115390.65995443371
67724.81403164806
89867.3023762282
108840.53592196561


KeyboardInterrupt: 



   
A class RF representing the random forest classifier. It could represent with methods:

subsampling from the dataset
build a model with multiple decision trees
apply the model
    

    

    

    

    
    
    

In [153]:
class RF(object):
    def __init__(self):
        self.data = None  # training data set (loaded into memory)
        self.trees = None  # decision trees
    
    def __load_data(self):
        pass
        
    def __subsampling(self, train_set, sample_size_ratio):
        sample_number = round(len(self.data) * sample_size_ratio)
        # TODO: generate a subsample with replacement
    def build_model(self, train_set, split_conditions, sample_size_ratio, number_of_trees):
        for i in range(number_of_trees):
            sample = self.__subsampling(train_set, sample_size_ratio)
            tree = DT() # build a tree with sample data and split conditions
            self.trees.append(tree)

    def predict(self, test_set):
        rf_predictions = []
        for row in test_set:
            predictions = [tree.predict(row) for tree in self.trees]
            rf_predictions.append(np.mean(predictions))
        return rf_predictions

In [154]:
def main():
    dt = DT()
    dt.build_model()

if __name__ == "__main__":
    main()

2
###################
1
15
15
{160, 70, 40, 75, 45, 80, 50, 20, 85, 180, 30, 120, 90, 60, 190}
###################
2
5
5
{0, 1, 2, 3, 4}
2
2
###################
5
2
2
{0, 1}
###################
6
3
3
{0, 1, -1}
###################
7
4
4
{0, 1, 2, 3}
###################
8
4
4
{0, 1, 2, 3}
###################
9
2
2
{0, 1}
###################
10
5
5
{0, 1, 2, 3, 4}
###################
11
3
3
{0, 1, 2}
###################
12
25
25
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24}
###################
13
9
9
{0, 1, 2, 3, 4, 5, 6, 7, 8}
###################
14
8
8
{0, 1, 2, 3, 4, 5, 6, 7}
###################
15
5
5
{0, 1, 2, 3, 4}
###################
16
8
8
{0, 1, 2, 3, 4, 5, 6, 7}
###################
17
10
10
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
###################
18
9
9
{1, 2, 3, 4, 5, 6, 7, 8, 9}
2
2
###################
21
6
6
{0, 1, 2, 3, 4, 5}
###################
22
8
8
{0, 1, 2, 3, 4, 5, 6, 7}
###################
23
15
15
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9